In [1]:
%cd ~/gymnasium-test/
%load_ext line_profiler

/home/n.saumik/gymnasium-test


In [2]:
import os
import time

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ["MPLCONFIGDIR"] = "/tmp/"
import tensorflow as tf

tf.config.set_visible_devices([], "GPU")

import numpy as np
from tqdm import tqdm

2024-09-06 03:31:17.014631: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [3]:
from tree import Forest, make_model

In [ ]:
def make_model(depth, opt="sgd", lr=0.01, num_nodes=16, num_layers=1):
    import tensorflow as tf

    inputs = tf.keras.layers.Input(shape=(2**depth - 1) * 2)
    x = tf.keras.layers.Flatten()(inputs)
    for _ in range(num_layers):
        x = tf.keras.layers.Dense(num_nodes, activation="relu")(x)
    output1 = tf.keras.layers.Dense(2, name="Y")(x)
    output1 = tf.keras.layers.Softmax()(output1)
    model = tf.keras.models.Model(inputs=inputs, outputs=output1)

    if opt == "adam":
        opt = tf.keras.optimizers.Adam(learning_rate=0.001 * lr)
    elif opt == "adamw":
        opt = tf.keras.optimizers.Adamw(learning_rate=0.001 * lr)
    elif opt == "sgd":
        opt = tf.keras.optimizers.SGD(
            learning_rate=0.01 * lr, momentum=0.90, nesterov=True
        )
    else:
        raise ValueError("Optimizer should be either 'adam' or 'sgd'")
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    model.compile(optimizer=opt, loss=loss_fn, metrics=["accuracy"], run_eagerly=False)
    return model

In [139]:
def exp(depth, mode, budget, seed, skill):
    X, Y = Forest(
        depth,
        mode,
        budget * seed,
        budget * (seed + 1),
        hide=True,
    ).get_training_data(skill)
    X = tf.convert_to_tensor(X, dtype=tf.float32)
    Y = tf.convert_to_tensor(Y, dtype=tf.float32)
    ds = (
        tf.data.Dataset.from_tensor_slices((X, Y))
        .cache()
        .shuffle(1000, seed)
        .batch(256)
        .prefetch(tf.data.experimental.AUTOTUNE)
    )

    test_budget, test_offset = 10000, int(1e9)
    testing = Forest(
        depth,
        mode,
        test_budget * seed + test_offset,
        test_budget * (seed + 1) + test_offset,
    )
    X_test, Y_test = testing.get_training_data(skill)
    X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
    Y_test = tf.convert_to_tensor(Y_test, dtype=tf.float32)
    X_test.shape, Y_test.shape

    st = time.perf_counter()
    model = make_model(depth, num_nodes=16, num_layers=2, opt="sgd", lr=10)
    # model.summary()
    # print("Time:", time.perf_counter() - st)

    accs = []
    accs_row = []
    scores = []

    for _ in tqdm(range(200)):
        st = time.perf_counter()
        model.fit(ds, verbose=0)
        # print("Time:", time.perf_counter() - st)

        st = time.perf_counter()
        Y_pred = model(X_test)
        acc = (Y_pred.numpy().argmax(axis=1) == Y_test.numpy()).astype(int)
        accs.append(acc.mean())
        accs_row.append([acc[i :: depth - 1].mean() for i in range(depth - 1)])
        # print(accs[-1], accs_row[-1])
        # model.evaluate(ds_test, steps=None)
        # print("Time:", time.perf_counter() - st)

        st = time.perf_counter()
        _, _, score = testing.eval_model(model)
        scores.append(score)
        # print("Time:", time.perf_counter() - st)
        # print(score)
        # print()
    accs, accs_row, scores = np.array(accs), np.array(accs_row), np.array(scores)
    return (accs_row[accs.argmax()],)

In [140]:
for mode in ["float", "binary"]:
    for depth in [4, 6, 8]:
        for skill in [2, 4, 6, 8]:
            if skill > depth:
                continue
            print(mode, depth, skill, exp(depth, mode, 1000, 0, skill))

100%|██████████| 200/200 [00:31<00:00,  6.42it/s]


float 4 2 (array([0.9842, 0.967 , 0.941 ]),)


100%|██████████| 200/200 [00:32<00:00,  6.17it/s]


float 4 4 (array([0.9319, 0.9445, 0.9511]),)


100%|██████████| 200/200 [00:48<00:00,  4.10it/s]


...

binary 8 4 (array([0.6857, 0.6575, 0.6505, 0.6579, 0.655 , 0.6567, 0.6988]),)


100%|██████████| 200/200 [01:28<00:00,  2.25it/s]


binary 8 6 (array([0.6703, 0.6316, 0.6173, 0.613 , 0.6165, 0.6316, 0.682 ]),)


100%|██████████| 200/200 [01:19<00:00,  2.51it/s]

binary 8 8 (array([0.6663, 0.6192, 0.5955, 0.5935, 0.6027, 0.6218, 0.6787]),)


In [159]:
testing.trees[0].get_all_paths(0, 3)

[[0, 1, 3], [0, 1, 4], [0, 2, 5], [0, 2, 6]]

In [163]:
testing = Forest(
    4,
    "float",
    test_budget * seed + test_offset,
    test_budget * (seed + 1) + test_offset,
)

In [167]:
testing.trees[0].get_all_paths(0, 3)

[[0, 1, 3], [0, 1, 4], [0, 2, 5], [0, 2, 6]]

In [168]:
testing.trees[0].n

15